# day 20

https://adventofcode.com/2018/day/20

In [ ]:
import os

import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns

import eri.logging as logging

In [ ]:
%matplotlib inline

sns.set()

In [ ]:
FNAME = os.path.join('data', 'day20.txt')

LOGGER = logging.getLogger('day20')
logging.configure()

logging.getLogger('matplotlib').setLevel(logging.WARN)

## part 1

### problem statement:

> While you were learning about instruction pointers, the Elves made considerable progress. When you look up, you discover that the North Pole base construction project has completely surrounded you.
> 
> The area you are in is made up entirely of rooms and doors. The rooms are arranged in a grid, and rooms only connect to adjacent rooms when a door is present between them.
> 
> For example, drawing rooms as ., walls as #, doors as | or -, your current position as X, and where north is up, the area you're in might look like this:
> 
>     #####
>     #.|.#
>     #-###
>     #.|X#
>     #####
>
> You get the attention of a passing construction Elf and ask for a map. "I don't have time to draw out a map of this place - it's huge. Instead, I can give you directions to every room in the facility!" He writes down some directions on a piece of parchment and runs off. In the example above, the instructions might have been `^WNE$`, a regular expression or "regex" (your puzzle input).
> 
> The regex matches routes (like WNE for "west, north, east") that will take you from your current room through various doors in the facility. In aggregate, the routes will take you through every door in the facility at least once; mapping out all of these routes will let you build a proper map and find your way around.
> 
> `^` and `$` are at the beginning and end of your regex; these just mean that the regex doesn't match anything outside the routes it describes. (Specifically, `^` matches the start of the route, and `$` matches the end of it.) These characters will not appear elsewhere in the regex.
> 
> The rest of the regex matches various sequences of the characters N (north), S (south), E (east), and W (west). In the example above, `^WNE$` matches only one route, WNE, which means you can move west, then north, then east from your current position. Sequences of letters like this always match that exact route in the same order.
> 
> Sometimes, the route can branch. A branch is given by a list of options separated by pipes (|) and wrapped in parentheses. So, `^N(E|W)N$` contains a branch: after going north, you must choose to go either east or west before finishing your route by going north again. By tracing out the possible routes after branching, you can determine where the doors are and, therefore, where the rooms are in the facility.
> 
> For example, consider this regex: `^ENWWW(NEEE|SSE(EE|N))$`
> 
> This regex begins with ENWWW, which means that from your current position, all routes must begin by moving east, north, and then west three times, in that order. After this, there is a branch. Before you consider the branch, this is what you know about the map so far, with doors you aren't sure about marked with a ?:
> 
>     #?#?#?#?#
>     ?.|.|.|.?
>     #?#?#?#-#
>         ?X|.?
>         #?#?#
> 
> After this point, there is (NEEE|SSE(EE|N)). This gives you exactly two options: NEEE and SSE(EE|N). By following NEEE, the map now looks like this:
> 
>     #?#?#?#?#
>     ?.|.|.|.?
>     #-#?#?#?#
>     ?.|.|.|.?
>     #?#?#?#-#
>         ?X|.?
>         #?#?#
> 
> Now, only SSE(EE|N) remains. Because it is in the same parenthesized group as NEEE, it starts from the same room NEEE started in. It states that starting from that point, there exist doors which will allow you to move south twice, then east; this ends up at another branch. After that, you can either move east twice or north once. This information fills in the rest of the doors:
> 
>     #?#?#?#?#
>     ?.|.|.|.?
>     #-#?#?#?#
>     ?.|.|.|.?
>     #-#?#?#-#
>     ?.?.?X|.?
>     #-#-#?#?#
>     ?.|.|.|.?
>     #?#?#?#?#
> 
> Once you've followed all possible routes, you know the remaining unknown parts are all walls, producing a finished map of the facility:
> 
>     #########
>     #.|.|.|.#
>     #-#######
>     #.|.|.|.#
>     #-#####-#
>     #.#.#X|.#
>     #-#-#####
>     #.|.|.|.#
>     #########
> 
> Sometimes, a list of options can have an empty option, like (NEWS|WNSE|). This means that routes at this point could effectively skip the options in parentheses and move on immediately. For example, consider this regex and the corresponding map:
> 
> `^ENNWSWW(NEWS|)SSSEEN(WNSE|)EE(SWEN|)NNN$`
> 
>     ###########
>     #.|.#.|.#.#
>     #-###-#-#-#
>     #.|.|.#.#.#
>     #-#####-#-#
>     #.#.#X|.#.#
>     #-#-#####-#
>     #.#.|.|.|.#
>     #-###-###-#
>     #.|.|.#.|.#
>     ###########
> 
> This regex has one main route which, at three locations, can optionally include additional detours and be valid: (NEWS|), (WNSE|), and (SWEN|). Regardless of which option is taken, the route continues from the position it is left at after taking those steps. So, for example, this regex matches all of the following routes (and more that aren't listed here):
> 
>     ENNWSWWSSSEENEENNN
>     ENNWSWWNEWSSSSEENEENNN
>     ENNWSWWNEWSSSSEENEESWENNNN
>     ENNWSWWSSSEENWNSEEENNN
> 
> By following the various routes the regex matches, a full map of all of the doors and rooms in the facility can be assembled.
> 
> To get a sense for the size of this facility, you'd like to determine which room is furthest from you: specifically, you would like to find the room for which the shortest path to that room would require passing through the most doors.
> 
> + In the first example (`^WNE$`), this would be the north-east corner 3 doors away.
> + In the second example (`^ENWWW(NEEE|SSE(EE|N))$`), this would be the south-east corner 10 doors away.
> + In the third example (`^ENNWSWW(NEWS|)SSSEEN(WNSE|)EE(SWEN|)NNN$`), this would be the north-east corner 18 doors away.
>
> Here are a few more examples:
> 
> `Regex: ^ESSWWN(E|NNENN(EESS(WNSE|)SSS|WWWSSSSE(SW|NNNE)))$`
>
>     Furthest room requires passing 23 doors
>     
>     #############
>     #.|.|.|.|.|.#
>     #-#####-###-#
>     #.#.|.#.#.#.#
>     #-#-###-#-#-#
>     #.#.#.|.#.|.#
>     #-#-#-#####-#
>     #.#.#.#X|.#.#
>     #-#-#-###-#-#
>     #.|.#.|.#.#.#
>     ###-#-###-#-#
>     #.|.#.|.|.#.#
>     #############
> 
>     Regex: ^WSSEESWWWNW(S|NENNEEEENN(ESSSSW(NWSW|SSEN)|WSWWN(E|WWS(E|SS))))$
>     Furthest room requires passing 31 doors
>     
>     ###############
>     #.|.|.|.#.|.|.#
>     #-###-###-#-#-#
>     #.|.#.|.|.#.#.#
>     #-#########-#-#
>     #.#.|.|.|.|.#.#
>     #-#-#########-#
>     #.#.#.|X#.|.#.#
>     ###-#-###-#-#-#
>     #.|.#.#.|.#.|.#
>     #-###-#####-###
>     #.|.#.|.|.#.#.#
>     #-#-#####-#-#-#
>     #.#.|.|.|.#.|.#
>     ###############
> 
> What is the largest number of doors you would be required to pass through to reach a room? That is, find the room for which the shortest path from your starting location to that room would require passing through the most doors; what is the fewest doors you can pass through to reach it?

#### loading data

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

#### function def

In [ ]:
import networkx as nx

D = {'N': 1, 'E': 1j, 'S': -1, 'W': -1j}

def path_to_network(paths):
    maze = nx.Graph()
    
    outer_shell = {0}
    group_stack = []
    shell_starts = {}
    shell_ends = set()

    for c in paths:
        if c == '|':
            # within a group, a split. our current position is a new group
            # ending position, so add that. then, re-boot the starting positions
            # to the start-of-group starting_positions
            shell_ends.update(outer_shell)
            outer_shell = shell_starts
        elif c in 'NESW':
            # move in a given direction: add all edges and update our current positions
            direction = D[c]
            maze.add_edges_from((p, p + direction) for p in outer_shell)
            outer_shell = {p + direction for p in outer_shell}
        elif c == '(':
            # start of group: add current positions as start of a new group
            group_stack.append((shell_starts, shell_ends))
            shell_starts, shell_ends = outer_shell, set()
        elif c == ')':
            # end of group: finish current group, add current positions as possible ends
            outer_shell.update(shell_ends)
            shell_starts, shell_ends = group_stack.pop()

    return maze

In [ ]:
def maze_pos(maze):
    return {n: (n.imag, n.real) for n in maze.nodes()}

In [ ]:
def vis_maze(maze, figsize=(6, 6), **kwargs):
    fig, ax = plt.subplots(figsize=figsize)
    nx.draw(maze, pos=maze_pos(maze), ax=ax, **kwargs)

In [ ]:
def q_1(data):
    maze = path_to_network(data)
    
    lengths = nx.algorithms.shortest_path_length(maze, 0)
    return max(lengths.values())

#### tests

In [ ]:
test_0, val_0 = '^WNE$', 3
test_1, val_1 = '^ENWWW(NEEE|SSE(EE|N))$', 10
test_2, val_2 = '^ENNWSWW(NEWS|)SSSEEN(WNSE|)EE(SWEN|)NNN$', 18
test_3, val_3 = '^ESSWWN(E|NNENN(EESS(WNSE|)SSS|WWWSSSSE(SW|NNNE)))$', 23
test_4, val_4 = '^WSSEESWWWNW(S|NENNEEEENN(ESSSSW(NWSW|SSEN)|WSWWN(E|WWS(E|SS))))$', 31

In [ ]:
maze_0 = path_to_network(test_0)
maze_0.nodes()

In [ ]:
vis_maze(path_to_network(test_0))

In [ ]:
vis_maze(path_to_network(test_1))

In [ ]:
vis_maze(path_to_network(test_2))

In [ ]:
vis_maze(path_to_network(test_3))

In [ ]:
vis_maze(path_to_network(test_4))

In [ ]:
vis_maze(path_to_network(load_data()), figsize=(15, 15), node_size=10)

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_0) == val_0
    assert q_1(test_1) == val_1
    assert q_1(test_2) == val_2
    assert q_1(test_3) == val_3
    assert q_1(test_4) == val_4
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

> Okay, so the facility is big.
>
> How many rooms have a shortest path from your current location that pass through at least 1000 doors?

#### function def

In [ ]:
def q_2(data):
    maze = path_to_network(data)
    
    lengths = nx.algorithms.shortest_path_length(maze, 0)
    return sum(l >= 1000 for l in lengths.values())

#### answer

In [ ]:
q_2(load_data())

fin